In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from datetime import datetime, timezone
import pandas as pd

current_date = pd.to_datetime(datetime.now(timezone.utc)).floor('H')
print(f'{current_date = }')

current_date = Timestamp('2025-01-18 16:00:00+0000', tz='UTC')


In [11]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

2025-01-18 21:48:31,149 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-18 21:48:31,166 INFO: Initializing external client
2025-01-18 21:48:31,166 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-18 21:48:33,237 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207467
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (13.91s) 
ts_data =                      pickup_hour  pickup_location_id  rides
152769 2024-12-21 16:00:00+00:00                   1    2.0
24563  2024-12-21 17:00:00+00:00                   1    3.0
131967 2024-12-21 18:00:00+00:00                   1    7.0
48472  2024-12-21 19:00:00+00:00                   1    1.0
112310 2024-12-21 20:00:00+00:00                   1    1.0
...                          ...                 ...    ...
106145 2025-01-18 11:00:00+00:00                 263  162.0
154397 2025-01-18 12:00:00+00:00                 263 

In [12]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)
predictions.head()

2025-01-18 21:49:07,627 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-18 21:49:07,636 INFO: Initializing external client
2025-01-18 21:49:07,636 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-18 21:49:09,557 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207467
[LightGBM] [Warning] feature_fraction is set=0.9528745323326657, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9528745323326657
[LightGBM] [Warning] bagging_fraction is set=0.96864818553338, subsample=1.0 will be ignored. Current value: bagging_fraction=0.96864818553338


,pickup_location_id,predicted_demand
0,208,1.0
1,90,94.0
2,181,1.0
3,63,2.0
4,147,1.0


In [13]:
predictions['pickup_hour'] = current_date
predictions.head()

,pickup_location_id,predicted_demand,pickup_hour
0,208,1.0,2025-01-18 16:00:00+00:00
1,90,94.0,2025-01-18 16:00:00+00:00
2,181,1.0,2025-01-18 16:00:00+00:00
3,63,2.0,2025-01-18 16:00:00+00:00
4,147,1.0,2025-01-18 16:00:00+00:00


In [15]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

2025-01-18 21:51:22,408 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-18 21:51:22,411 INFO: Initializing external client
2025-01-18 21:51:22,414 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-18 21:51:24,371 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207467


In [16]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207467/fs/1196121/fg/1395015


Uploading Dataframe: 100.00% |██████████| Rows 260/260 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207467/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(Job('model_predictions_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)